In [ ]:
from google.colab import  files
file = files.upload()

Saving best_model_90.pth to best_model_90.pth


In [ ]:
import kagglehub

path = kagglehub.dataset_download("briscdataset/brisc2025")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'brisc2025' dataset.
Path to dataset files: /kaggle/input/brisc2025


In [ ]:
import shutil

In [ ]:
shutil.copytree("/kaggle/input/brisc2025/brisc2025/classification_task", "/content/classification_task")

'/content/classification_task'

# **LIBRARY**

In [ ]:
import torch
import torchvision
from torchvision import datasets, transforms
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import Adam
import torchvision.models as model
import matplotlib.pyplot as plt
import numpy as np

torch.manual_seed(42)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
import random, numpy as np
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

DATA

In [ ]:
#augment
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=2),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(5),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std= [0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.75,
                            scale=(0.01, 0.3),
                            ratio=(1.0, 1.0),
                            value=0,
                            inplace =True)
])
test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std= [0.229, 0.224, 0.225]),
])
#dataset
dataset_train = datasets.ImageFolder('classification_task/train',train_transform)
dataset_test = datasets.ImageFolder('classification_task/test',test_transform)
#loader
trainloader = DataLoader(dataset_train,batch_size=32,shuffle=True)
testloader = DataLoader(dataset_test,batch_size=32,shuffle=False)

MODEL


In [ ]:
class CNNModel(nn.Module):
    def __init__(self, n_classes=4, dropout_rate=0.3):
        super(CNNModel, self).__init__()
        self.conv_layer1 = nn.Sequential(
            nn.Conv2d(3, 128, 3, stride=1, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(128)

        )
        self.conv_layer2 = nn.Sequential(
            nn.Conv2d(128, 128, 3, stride=1, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2, 2),
            nn.Dropout(dropout_rate)  # Dropout after layer 2
        )
        self.skip1 = nn.Sequential(
            nn.Conv2d(3, 128, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128)
        )



        self.conv_layer3 = nn.Sequential(
            nn.Conv2d(128, 256, 3, stride=1, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(256)
        )
        self.conv_layer4 = nn.Sequential(
            nn.Conv2d(256, 256, 3, stride=1, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2, 2),
            nn.Dropout(dropout_rate)  # Dropout after layer 4
        )
        self.skip2 = nn.Sequential(
            nn.Conv2d(128, 256, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256)
        )



        self.conv_layer5 = nn.Sequential(
            nn.Conv2d(256, 512, 3, stride=1, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(512)
        )
        self.conv_layer6 = nn.Sequential(
            nn.Conv2d(512, 512, 3, stride=1, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(512)
        )
        self.conv_layer7 = nn.Sequential(
            nn.Conv2d(512, 512, 3, stride=1, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(512),
            nn.MaxPool2d(2, 2),
            nn.Dropout(dropout_rate)  # Dropout after layer 7
        )
        self.skip3 = nn.Sequential(
            nn.Conv2d(256, 512, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(512)
        )



        self.conv_layer8 = nn.Sequential(
            nn.Conv2d(512, 1024, 3, stride=1, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(1024)
        )
        self.conv_layer9 = nn.Sequential(
            nn.Conv2d(1024, 1024, 3, stride=1, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(1024)
        )
        self.conv_layer10 = nn.Sequential(
            nn.Conv2d(1024, 1024, 3, stride=1, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(1024),
            nn.MaxPool2d(2, 2),
            nn.Dropout(dropout_rate)  # Dropout after layer 10
        )
        self.skip4 = nn.Sequential(
            nn.Conv2d(512, 1024, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(1024)
        )



        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1024*14*14, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Linear(512, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Linear(64, n_classes)
        )

    def forward(self, x):
        skip = self.skip1(x)
        x = self.conv_layer1(x)
        x = self.conv_layer2(x)
        #print(f"After conv_layer2: {x.shape}")
        x = x + skip

        skip = self.skip2(x)
        x = self.conv_layer3(x)
        x = self.conv_layer4(x)
        #print(f"After conv_layer4: {x.shape}")
        x = x + skip

        skip = self.skip3(x)
        x = self.conv_layer5(x)
        x = self.conv_layer6(x)
        x = self.conv_layer7(x)
        #print(f"After conv_layer7: {x.shape}")
        x = x + skip

        skip = self.skip4(x)
        x = self.conv_layer8(x)
        x = self.conv_layer9(x)
        x = self.conv_layer10(x)
        #print(f"After conv_layer10: {x.shape}")
        x = x + skip

        out = self.classifier(x)

        return out

In [ ]:
model = CNNModel(n_classes=4)   # khởi tạo model với cùng kiến trúc
checkpoint = torch.load("best_model_90.pth", map_location=device)
model.load_state_dict(checkpoint)
model.to(device)

CNNModel(
  (conv_layer1): Sequential(
    (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_layer2): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.3, inplace=False)
  )
  (skip1): Sequential(
    (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_layer3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_layer4): Se

In [ ]:
from torchsummary import summary

summary(model, input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [-1, 128, 112, 112]           3,584
              ReLU-2        [-1, 128, 112, 112]               0
       BatchNorm2d-3        [-1, 128, 112, 112]             256
            Conv2d-4        [-1, 128, 224, 224]           3,584
              ReLU-5        [-1, 128, 224, 224]               0
       BatchNorm2d-6        [-1, 128, 224, 224]             256
            Conv2d-7        [-1, 128, 224, 224]         147,584
              ReLU-8        [-1, 128, 224, 224]               0
       BatchNorm2d-9        [-1, 128, 224, 224]             256
        MaxPool2d-10        [-1, 128, 112, 112]               0
          Dropout-11        [-1, 128, 112, 112]               0
           Conv2d-12          [-1, 256, 56, 56]         295,168
             ReLU-13          [-1, 256, 56, 56]               0
      BatchNorm2d-14          [-1, 256,

# **TRAIN**

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=1e-4)
decayRate = 0.97
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decayRate)

In [ ]:
# Function to compute loss and accuracy for test set
def evaluate(model, testloader, criterion):
    model.eval()
    test_loss = 0.0
    running_correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader:
            # Move inputs and labels to the device
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            running_correct += (predicted == labels).sum().item()

    accuracy = 100 * running_correct / total
    test_loss = test_loss / len(testloader)
    return test_loss, accuracy

In [ ]:
# some parameter
train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []
max_epoch = 250

In [ ]:
# train
for epoch in range(max_epoch):
    model.train()
    running_loss = 0.0
    running_correct = 0   # to track number of correct predictions
    total = 0             # to track total number of samples

    for i, (inputs, labels) in enumerate(trainloader, 0):
        # Move inputs and labels to the device
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        running_loss += loss.item()

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Determine class predictions and track accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        running_correct += (predicted == labels).sum().item()


    epoch_accuracy = 100 * running_correct / total
    epoch_loss = running_loss / (i + 1)

    test_loss, test_accuracy = evaluate(model, testloader, criterion)
    print(f"Epoch [{epoch + 1}/{max_epoch}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")

    # save for plot
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_accuracy)
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)

    # update learning rate
    lr_scheduler.step()

Epoch [1/250], Loss: 2.6285, Accuracy: 35.58%, Test Loss: 0.6332, Test Accuracy: 79.80%
Epoch [2/250], Loss: 2.1063, Accuracy: 35.22%, Test Loss: 0.6695, Test Accuracy: 76.40%
Epoch [3/250], Loss: 1.8655, Accuracy: 36.10%, Test Loss: 0.4070, Test Accuracy: 82.90%
Epoch [4/250], Loss: 1.6708, Accuracy: 35.62%, Test Loss: 0.4900, Test Accuracy: 80.60%
Epoch [5/250], Loss: 1.5458, Accuracy: 37.78%, Test Loss: 0.4926, Test Accuracy: 80.80%
Epoch [6/250], Loss: 1.5174, Accuracy: 36.76%, Test Loss: 0.4761, Test Accuracy: 81.00%
Epoch [7/250], Loss: 1.4588, Accuracy: 37.86%, Test Loss: 0.4375, Test Accuracy: 82.90%
Epoch [8/250], Loss: 1.4356, Accuracy: 38.54%, Test Loss: 0.4465, Test Accuracy: 82.50%
Epoch [9/250], Loss: 1.3872, Accuracy: 38.78%, Test Loss: 0.4712, Test Accuracy: 82.80%
Epoch [10/250], Loss: 1.3617, Accuracy: 39.38%, Test Loss: 0.4706, Test Accuracy: 83.20%
Epoch [11/250], Loss: 1.3398, Accuracy: 39.64%, Test Loss: 0.4925, Test Accuracy: 82.50%
Epoch [12/250], Loss: 1.3437, 

KeyboardInterrupt: 

# **METRICS**

In [ ]:
checkpoint = torch.load("best_model_90.pth", map_location=device)
model.load_state_dict(checkpoint)
model.to(device)

CNNModel(
  (conv_layer1): Sequential(
    (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_layer2): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.3, inplace=False)
  )
  (skip1): Sequential(
    (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_layer3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_layer4): Se

In [ ]:
from sklearn.metrics import classification_report
import torch

def evaluate_full_metrics(model, testloader, criterion, class_names, device):
    model.eval()
    test_loss = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in testloader:
            # Move data to device
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            # Lấy dự đoán
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Trung bình loss
    test_loss = test_loss / len(testloader)

    # Báo cáo chi tiết các metric
    report = classification_report(
        all_labels,
        all_preds,
        target_names=class_names,
        digits=2
    )

    # Accuracy chung
    accuracy = (torch.tensor(all_preds) == torch.tensor(all_labels)).float().mean().item() * 100

    print(report)
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Overall Accuracy: {accuracy:.2f}%")

    return test_loss, accuracy, report


In [ ]:
test_loss, test_acc, report = evaluate_full_metrics(
    model, testloader, criterion,
    class_names=['glioma', 'meningioma', 'no_tumor', 'pituitary'],
    device=device
)

              precision    recall  f1-score   support

      glioma       0.94      0.80      0.86       254
  meningioma       0.84      0.90      0.87       306
    no_tumor       0.89      0.97      0.93       140
   pituitary       0.95      0.97      0.96       300

    accuracy                           0.91      1000
   macro avg       0.91      0.91      0.91      1000
weighted avg       0.91      0.91      0.91      1000

Test Loss: 67.5466
Overall Accuracy: 90.60%
